# SI 618: Data Manipulation and Analysis
## 03 - Advanced pandas
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a> This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

### IMPORTANT: Replace ```?``` in the following code with your uniqname.

In [ ]:
MY_UNIQNAME = '?'

## Before we start...
## <font color="magenta">Q0: (2 point) Please let us know what you found confusing in the last class. </font>
We'll try to take time in the next class to review these concepts next class.

# Loading and manipulating data in pandas

## Learning Objectives
* load CSV files
* load JSON files
* use pd.read_html to extract tables from web pages
* load data from simple APIs 
* load data from a SQL database
* handle missing data (dropna and fillna)
* use vectorized string functions
* Pandas refresher (or introduction)
* explain how pandas operations differ from "traditional" python
* be able to load a CSV file into a Pandas DataFrame
* explain how to extract columns from a DataFrame
* sort a DataFrame
* assign a column as the index of a DataFrame
* filter a DataFrame according to some criteria
* explain how boolean masks work in filtering DataFrames

In [ ]:
import pandas as pd

Recall the ```pd.read_csv``` function that we used to load data sets in previous classes:

In [ ]:
menu = pd.read_csv('data/menu.csv') 

That works great for well-formatted CSV files, but what happens when you get something that looks like the ```data/avocado_eu.csv``` file.
Go ahead and browse that in JupyterLab's CSV browser.

You'll notice a new drop-down menu labelled "Delimiter".  Go ahead and change that to ```;```.

Referring back to your readings and the [read_csv documentation online](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html), complete the following exercise


Read the data/avocado_eu.csv file into a pandas DataFrame and show the first 5 rows.


In [ ]:
avocado = pd.read_csv('data/avocado_eu.csv', delimiter=';')
avocado.head(5)

You'll notice that, unless you did something special in the previous read_csv invocation, the decimal points don't look quite right.  Go ahead and find the right option to convert commas to periods when loading a CSV file.

## <font color="magenta">Q1 (2 point): 
Read the data/avocado_eu.csv file using the correct delimiter and decimal character into a dataframe and show the first 5 rows: </font>

In [ ]:
#Insert your code here

# Counting the number of values

Sometimes, you'll want to count the number of times values occur.  For example, we might want to know the number of times each 'type'
is reported in our avocado data.  Use the ```value_counts()``` function on a Series to do so:

In [ ]:
avocado['type'].value_counts()

# Loading JSON data

In addition to CSV files, JSON (JavaScript Object Notation) files or data is commonly used.  

In [ ]:
nfl_football_players = pd.read_json('data/nfl_football_profiles.json')

In [ ]:
nfl_football_players.head()

And, just for fun, show the player with the highest Current Salary from that dataset:

In [ ]:
nfl_football_players.sort_values('current_salary', ascending=False).head(1)

# Fixing up the data
Assuming you did something like sort_values on one of the original columns, you probably got the wrong result.

Looking a bit more closely at the results, you'll notice that the current_salary column.  Remembering that we have made the shift from pythonic to pandorable, we can leverage the impressive-sounding "vectorized string functions" mentioned in Section 7.3 of the McKinney book.  Specifically, we can use the str.replace(...) method.  Note that had we use read_csv to load the file we could have used the ```thousands=``` option and avoided all this, but sometimes data doesn't come in a convenient format.

One way to apply functions is to operate on a column and then assign the results to another column.  For example, if we wanted to eliminate commas, we could replace them with null strings

In [ ]:
nfl_football_players['current_salary'].str.replace(',', '')

And assign the results to a column in the original dataframe (in this case I'm calling the column current_salary_nocommas). We need to assign it to a column to ensure the operation is done in-place.

In [ ]:
nfl_football_players['current_salary_nocommas'] = nfl_football_players['current_salary'].str.replace(',', '')

But you'll notice that the type of the column is string, and we want to convert it to a float so we can sort it numerically.  So we can use the astype() function to convert it:

In [ ]:
nfl_football_players['current_salary_cleaned'] = nfl_football_players['current_salary_nocommas'].astype(float)

In [ ]:
nfl_football_players.head(2)

And now we can re-run our command to sort by salary and get the correct result:

In [ ]:
nfl_football_players.sort_values('current_salary_cleaned', ascending=False).head(1)

# Dropping missing values

In addition to the "all" or "any" functionality described in McKinney section 7.1, it's sometimes useful to drop a row only if a certain column or columns have missing data.  To do this, use the subset= option with dropna().  So, for example, to drop all players for whom we do not have salary information, we could use the following code:

In [ ]:
nfl_football_players.current_salary_cleaned.isna().sum()

In [ ]:
nfl_football_players_salaries = nfl_football_players.dropna(subset=['current_salary_cleaned'])

In [ ]:
nfl_football_players_salaries.head()

# Creating dummy variables

We might, on occasion, want to "bin" or "discretize" a variable.  For example, we might want to take the previous dataframe and add dummy variables that map onto whether the salaries are "small" (< \\$1M) , "medium" (\\$1M - \\$10M), or "large" (> \\$10M). A **dummy variable** is a binary indicator variable (often used in regressions) to represent a categorical variable. We could do something like the following:

In [ ]:
bins = [0,1000000,10000000,1000000000]

In [ ]:
pd.cut(nfl_football_players_salaries['current_salary_cleaned'],bins,labels=['small','medium','large'])[0:10]

In [ ]:
dummies = pd.get_dummies(pd.cut(nfl_football_players_salaries['current_salary_cleaned'],bins,labels=['small','medium','large']))

In [ ]:
dummies.head()

pd.concat stacks together objects along an axis _(Section 8.2 McKinney)_

In [ ]:
nfl_cats = pd.concat([nfl_football_players_salaries,dummies],axis=1)

In [ ]:
nfl_cats.tail()

# Scraping Tables from HTML

The ```pd.read_html``` function returns a list of DataFrames read from an HTML source.  The following line will return a list of DataFrames from https://en.wikipedia.org/wiki/List_of_largest_sports_contracts

In [ ]:
contracts_scraped = pd.read_html('https://en.wikipedia.org/wiki/List_of_largest_sports_contracts',header=0)

In [ ]:
len(contracts_scraped)

To get the first table, you'll need to pull off the 0th element:

In [ ]:
contracts = contracts_scraped[0]
contracts.head()

## <font color="magenta"> Q2 (2 points): 
Count the number of players from each sport in the List of Largest Sports Contracts. 
Hint:  see value_counts() description above
<font>

In [ ]:
#Insert your code here

_Insert English explanation of findings here._

## <font color="magenta">Q3 (12 points): 
Create a new dataframe that contains all the columns in the nfl_football_players dataframe as well as an additional column that contains each player's height in centimeters. Show the first 5 rows of your result. </font>

**Hints:** 
- 1 inch = 2.54 cm
- you can use the vectorized string function str.split() to separate feet and inches from the original dataframe column
- remember to cast strings to numeric types if you're going to perform math on them
- you might want to create an intermediate (temporary) DataFrame to help you keep things clear instead of attempting to do 
this in one line

In [ ]:
#Insert Code Here. You should use multiple code boxes. You can add them below.

# Joining, Combining, and Reshaping

## Learning Objectives
* use pd.read_html's parameters to extract specific tables from web pages
* create dataframes from lists and dictionaries
* use Pandas' apply function to run a function on each row of a dataframe
* view and set the indexes of a dataframe, including hierarchical indexes
* use loc to explore a dataframe with hierarchical indexes
* use stack and unstack to reshape dataframes
* concatenate two DataFrames by columns
* rename a dataframe's columns with a dictionary
* use Pandas' merge functionn to join dataframes in a SQL-like way

In [ ]:
import pandas as pd

In [ ]:
frodo_url = 'https://en.wikipedia.org/wiki/Frodo_Baggins'
frodo_tables = pd.read_html(frodo_url)
frodo_tables[0]

Now let's load the page for [Legolas](https://en.wikipedia.org/wiki/Legolas):

In [ ]:
legolas_url = 'https://en.wikipedia.org/wiki/Legolas'
legolas_tables = pd.read_html(legolas_url)
legolas_tables[0]

Hmmmm.  That doesn't look quite right.

Let's take a look at some URLs and figure out what's going on:

In [ ]:
pd.show_versions()

### Inspect the Frodo and Legolas pages and see if you can figure out some _attributes_ of the table we're interested in.


Describe what you found.

You'll notice that there are some characteristics that the "Information" box share across pages.  We can leverage that 
information by using the ```attrs``` attribute of ```read_html```.  For example, if we wanted to extract  the element(s) that had
an ```id``` of ```info```, we could use

```pd.read_html(url,{'id':'info'})```



## <font color="magenta">Q4: (2 point) Fill in the following code block to extract only the "Information" table for the Legolas page:

In [ ]:
a = {} # create an appropriate dictionary
pd.read_html(legolas_url, attrs=a)

Now let's define a function that, given a Wikipedia URL, will extract the contents of the Aliases component of the infobox table:

In [ ]:
def get_aliases(url):
    tables = pd.read_html(url, attrs={'class':'infobox'}) # extract only tables with class=infobox
    print(url,len(tables))   # sanity check: we should have just 1 table
    infotable = tables[0]    # pull the first table into a DataFrame
    ret = ''                 # initialize an empty string for our return value
    try:                     # in case the next line throws an exception
        x = infotable.set_index(0).loc['Aliases'] # setting the index on column 0 will allow us to use .loc to look up the value of 'Aliases'
        ret = x.values[0]
    except:
        ret = 'None'
    return ret

And let's try it out:

In [ ]:
get_aliases(legolas_url)

So far, so good.  It seems to work.  Now let's set up a DataFrame with a bunch of URLs:

In [ ]:
urls = ['https://en.wikipedia.org/wiki/Gimli_(Middle-earth)',
        'https://en.wikipedia.org/wiki/Frodo_Baggins',
        'https://en.wikipedia.org/wiki/Legolas',
        'https://en.wikipedia.org/wiki/Bilbo_Baggins',
        'https://en.wikipedia.org/wiki/Samwise_Gamgee',
        'https://en.wikipedia.org/wiki/Peregrin_Took',
        'https://en.wikipedia.org/wiki/Boromir',
        'https://en.wikipedia.org/wiki/Galadriel',
        'https://en.wikipedia.org/wiki/Meriadoc_Brandybuck']
names = ['Gimli',
         'Frodo',
         'Legolas',
         'Bilbo',
         'Sam',
         'Pippin',
         'Boromir',
         'Galadriel',
         'Meriadoc']

In [ ]:
udf = pd.DataFrame()
udf['name'] = names
udf['url'] = urls

In [ ]:
udf

The pythonic way of iterating through each of those rows would involve the use of some sort of ```for``` loop.  In pandas,
however, as can use the ```apply``` function to process an entire column!

In [ ]:
udf['url'].apply(get_aliases)

We can take the resulting Series and assign it to a new column in our DataFrame:

In [ ]:
udf['aliases'] = udf['url'].apply(get_aliases)

In [ ]:
udf

Let's just put the ```udf``` DataFrame aside for now.  We'll return to it later.

## Creating DataFrames and Exploring Indexes

Let's load the usual libraries...

In [ ]:
import pandas as pd
import numpy as np

Let's create some lists of data that we can use to construct a DataFrame:

In [ ]:
names = ['Gandalf',
         'Gimli',
         'Frodo',
         'Legolas',
         'Bilbo',
         'Sam',
         'Pippin',
         'Boromir',
         'Aragorn',
         'Galadriel',
         'Meriadoc',
        'Lily']
races = ['Maia',
         'Dwarf',
         'Hobbit',
         'Elf',
         'Hobbit',
         'Hobbit',
         'Hobbit',
         'Man',
         'Man',
         'Elf',
         'Hobbit',
        'Hobbit']
magic = [10, 1, 4, 6, 4, 2, 0, 0, 2, 9, 0, np.NaN]
aggression = [7, 10, 2, 5, 1, 6, 3, 8, 7, 2, 4, np.NaN ]
stealth = [8, 2, 5, 10, 5, 4 ,5, 3, 9, 10, 6, np.NaN]

There are a few different ways to construct a DataFrame.  We can either use an empty constructor and assign Series:

## <font color="magenta"> Q5: (2 points) Construct a dataframe with 5 columns (names, races, magic, aggression, and stealth) using the lists above.

In [ ]:
df = #Insert your code here

In [ ]:
df

Alternatively, we could have set things up with a dict:

In [ ]:
df = pd.DataFrame({'name': names,'race':races,'magic':magic,'aggression': aggression,'stealth':stealth})

In [ ]:
df.head()

Let's take a look at the index on the resulting DataFrame:

In [ ]:
df.index

We can set the index to something more useful than the default RangeIndex:

In [ ]:
df_nameindexed = df.set_index('name')

And if we take a look at the results, we see that we have a pandas Index instead of a RangeIndex:

In [ ]:
df_nameindexed.index

In [ ]:
df_nameindexed

Setting the name Series as the index allows us to do things like:

In [ ]:
df_nameindexed.loc['Aragorn']

Now recall the Hierarchical indexing from the readings. _(Section 8.1 McKinney)_ We can pass a list of column names to set_index to create a Hierarchical Index:

In [ ]:
df_racename_indexed = df.set_index(['race','name'])

In [ ]:
df_racename_indexed.index

This will allow us to get a DataFrame that matches a value on the outer index:

In [ ]:
df_racename_indexed.loc['Hobbit']

We can also use the index on a Series to match the outer index:

In [ ]:
df_racename_indexed['magic'].loc['Hobbit']

Or both indexes:

In [ ]:
df_racename_indexed['magic'].loc['Hobbit','Frodo']

Or just the inner index:

In [ ]:
df_racename_indexed['magic'].loc[:,'Frodo']

## <font color="magenta"> Q6: (2 point) Using .loc find how much aggression Legalos, an Elf, has.

In [ ]:
# Insert your code here

## Stacking and Unstacking

Stacking takes "wide" data and makes it "taller"

In [ ]:
df.set_index(['race']).stack()

If we call reset_index on the resulting Series, we get the following DataFrame:

In [ ]:
df.set_index(['race']).stack().reset_index()

The column names in the above DataFrame aren't particularly helpful, so we can rename them:

In [ ]:
df.set_index(['race']).stack().reset_index().rename(columns = {'level_0':'ID','level_1':'variable',0:'value'})

You can do the opposite of stacking by using the ```unstack``` function:

In [ ]:
df_stacked = df.stack()

In [ ]:
df_stacked

In [ ]:
df_stacked.unstack()

Why would we want to stack or unstack?  It depends on what sorts of analyses we want to do "downstream".  It's also the basis for pivoting, melting, and pivot tables, which we'll cover in the next class.

## Joining Data



Let's say we have another CSV file that contains URLs to Wikipedia pages for some of the LOTR characters:

In [ ]:
urls = pd.read_csv('data/lotr_wikipedia.csv')
urls

In [ ]:
urls = pd.read_csv('data/lotr_wikipedia.csv', index_col=0)

In [ ]:
urls

Let's take a look at the original DataFrame:

It looks like the rows are "aligned", so we can use the ```concat``` function to concatenate the two DataFrames.
Note that we specify the axis to be the columns.  The default is to concatenate by rows, which isn't what we want.

In [ ]:
pd.concat([df,urls],axis="columns")

That's great, and it's consistent with what we've used in previous classes.  But what happens if the 
rows in the two DataFrames don't match up?  Let's load another file that has a slightly different
sequence of rows:

## <font color="magenta"> Q7: (2 point) Construct a dataframe with lotr_wikipedia_wrong_order.csv which is in the data folder and concat it with df.

In [ ]:
#Insert your code here

Take a closer look at the name and url columns.  Something's not quite right.

We can work around that by using the appropriate indexing and then using the SQL-like ```merge``` function.

In [ ]:
df_names = df.set_index('name')

In [ ]:
df_names

In [ ]:
urls_wrong_order_names = urls_wrong_order.set_index('name')

In [ ]:
df_names.join(urls_wrong_order_names)

In [ ]:
df.head()

In [ ]:
urls_wrong_order.head()

In [ ]:
urls_wrong_order['name']

In [ ]:
df['name']

In [ ]:
df.merge(urls_wrong_order,on='name')

Now let's add a few additional URLs:

In [ ]:
urls_extras = pd.read_csv("data/lotr_wikipedia_extras.csv")

In [ ]:
urls_extras

And now let's use concat to add the new entries to the DataFrame.

In [ ]:
urls_complete = pd.concat([urls,urls_extras])

In [ ]:
urls_complete

Now that we've got a complete (for our purposes) list of URLs, let's use that DataFrame and our original
one to demonstrate the different types of ```join```s.

By default, ```join``` uses a left join, which means the all the values from the "left"
side are used, whether or not there's a corresponding entry from the "right" side.  In the example 
below, note that the url value for "Lily" is "NaN":

In [ ]:
df.merge(urls_wrong_order,on='name',how='left')

The "opposite" of a left join is, perhaps unsurprisingly, a "right" join, in which
all the values from the "right" side are used, whether or not a corresponding
value from the "left" side exists. Note in the following example that "Lily" has
disappeared, and Treebeard and Elrond lack information about "race", "magic", "aggression", and "stealth".

In [ ]:
df.merge(urls_wrong_order,on='name',how='right')

In addition to "left" and "right" joins, we have "outer" joins, which include
values from both the "left" and "right" DataFrames, regardless of whether
there are corresponding values in the other DataFrame.  Note that all of 
"Lily", "Treebeard" and "Elrond" are present in the following DataFrame:

In [ ]:
df.merge(urls_wrong_order,on='name',how='outer')

Finally, there are "inner" joins, which include only those values that exist in both the "left" and "right" DataFrames:

In [ ]:
df.merge(urls_wrong_order,on='name',how='inner')

Sometimes it's nice to know how a particular row got added to the resulting DataFrame.  Using ```indicator=True```
allows us to examine this:

In [ ]:
df.merge(urls_wrong_order,how='outer',indicator=True)

You'll note that we used the ```merge``` function from the DataFrame and passed in the other DataFrame as an argument.
You can also call the ```merge``` function from pandas directly and pass it the two DataFrames you are merging:

![pivot 1](http://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png "pivots")

In [ ]:
pd.merge(df,urls_wrong_order,how='outer',indicator=True)

## <font color="magenta">Q8: (2 points) Join the ```udf``` DataFrame (that contains aliases) to the ```df``` DataFrame using an appropriate merge

In [ ]:
# Insert your code here

# Aggregation and Grouping

## Learning Objectives
* use StringIO to create a DataFrame
* use the .describe() function
* understand .groupby()
* know how to use pivot and pivot_table

In [ ]:
import pandas as pd
import numpy as np

## Some more basic pandas functionality
One of the nice things about pandas is that it simplifies many common operations on datasets.  Let's load yet another LOTR dataset, 
this time using StringIO.  StringIO allows us to create a string that's then available as a file!

Why would you want to do this?  Sometimes it's easier to just paste data right into a Jupyter notebook (or python script)
than it is to create another CSV file.


In [ ]:
from io import StringIO

LOTRDATA=StringIO("""name,race,gender,magic,aggression,stealth
Gandalf,Maia,Male,10.0,7.0,8.0
Gimli,Dwarf,None,1.0,10.0,2.0
Frodo,Hobbit,Male,4.0,2.0,5.0
Legolas,Elf,Male,6.0,5.0,10.0
Bilbo,Hobbit,Male,4.0,1.0,5.0
Sam,Hobbit,Male,2.0,6.0,4.0
Pippin,Hobbit,Male,0.0,3.0,5.0
Boromir,Human,Male,0.0,8.0,3.0
Aragorn,Human,Male,2.0,7.0,9.0
Galadriel,Elf,Female,9.0,2.0,10.0
Lily,Hobbit,Female,,,
Meriadoc,Hobbit,Male,,4.0,6.0
Melian,Maia,Female,10.0,5.0,9.0
Idril,Elf,Female,8.0,,8.0
""")

lotr = pd.read_csv(LOTRDATA, index_col=None)
lotr

Now that we have a DataFrame, we can get some basic statistics about it using the ```describe()``` function.
Note that ```describe()``` only returns values for numeric columns.  Note too that it returns another DataFrame

In [ ]:
lotr.describe()

In [ ]:
type(lotr.describe())

## Pivots/Stack/Unstack Intro
The following cells are based on: 
http://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/ , which is
one of the best guides to pivots, pivot tables, stacking and unstacking that I've encountered.

For demonstration purposes, let's create the same DataFrame that Nikolay Grozev uses in his tutorial:

In [ ]:
from collections import OrderedDict
from pandas import DataFrame

table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$', '2$', '3$', '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
metal = DataFrame(table)
metal

# create the table

In [ ]:
# make a table of items (rows) and costs (USD) 
# for each in gold and bronze
metal.pivot(index='Item',columns='CType',values='USD')

See the image below... we're telling Pandas to take the table above, create a row for every item. 
This is done by setting index to Item (the column in the original table that contains item names)
We then are telling pandas we want to create a column for every unique element in the
original CType column.  And finally, we want the value in the cells to be the value from the USD
column in the original table.

![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_simple1.png "pivots")

## <font color="magenta">Q9: (2 points) Using pivot and the dataframe below make a table with each person as the index, their purchased items as columns, and the price they paid as the values.</font>


In [ ]:
table1 = OrderedDict((
    ("Person", ['Scott', 'Julie', 'Shiyan', 'Julie','Scott','Julie', 'Shiyan', 'Julie','Scott', 'Julie']),
    ('Item Purchased',['Kit-Kat','Mango', 'Twix', 'M&Ms', 'Ferrero Rocher','Apple', 'Watermelon', 'Pineapple', 'Snickers','Pear']),
    ('Price Paid',  ['$1','$2', '$1.5', '$2', '$1.5','$1','$1','$2', '$1.5', '$2']),
    ))
fav = DataFrame(table1)
fav

In [ ]:
#Insert Code Here

In [ ]:
# end of exercise

In [ ]:
p = metal.pivot(index='Item',columns='CType',values='USD')  # same as above, I'm just renaming the pivot as "p"

Access the USD cost of Item0 for Gold customers...

First we find the row for Item0/Gold and then we select the USD column and pull out the value

In [ ]:
result = metal[((metal.Item == 'Item0') & (metal.CType == 'Gold'))].USD

In [ ]:
type(result)

In [ ]:
result

Do the same thing on pivoted table. Here we pull out the row for Item0, grab the Gold column and print the value

In [ ]:
p[p.index == 'Item0'].Gold.values

Now pivot by multiple columns, I want USD and EU prices. It returns a hierarchical index.

In [ ]:
metal.pivot(index='Item',columns='CType')

Access the USD cost of Item0 for Gold customers

In [ ]:
p = metal.pivot(index='Item',columns='CType')
p.USD[p.USD.index == 'Item0'].Gold.values

# What happens if there is a collision? 
See the problem?  There are two Item0/Golds:


![pivot 2](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_simple_error.png "pivots")

Let's set up another DataFrame to demonstrate this:

In [ ]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$', '2$', '3$', '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
metal = DataFrame(table)


In [ ]:
metal.head()

The next cell will generate an error:

In [ ]:
p = metal.pivot(index='Item', columns='CType', values='USD')
# will return an error

## pivot_tables is your friend
![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_table_simple1.png "pivots")

Let's create yet another DataFrame to play with:

In [ ]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1.1, 2.2, 3.3, 4.4])
))
metal = DataFrame(table)
metal

pivot_table is  a bit different than pivot... It's the same with the first part
index, columns, values remain the same as before BUT we added a rule (aggfunc)
that says: whey you hit a conflict, the way to resolve it is X (in this case
x is the "mean"... so find the mean of the two numbers)

In [ ]:
p = metal.pivot_table(index='Item',columns='CType',values='USD',aggfunc=np.mean)
p

You could have also resolved the conflict in other ways.  Here we tell it to take the "min":

In [ ]:
p = metal.pivot_table(index='Item',columns='CType',values='USD',aggfunc=np.min)
p

## <font color="magenta">Q10: (2 points) Using pivot_table and the dataframe below make a table with each person as the index, their purchased items as columns, and the sum of the price they paid as the values (sometimes people bought more than one of the same item).</font>


In [ ]:
table1 = OrderedDict((
    ("Person", ['Scott', 'Julie', 'Shiyan', 'Julie',
                'Scott','Julie', 'Shiyan', 'Julie','Scott', 
                'Julie','Scott', 'Julie', 'Shiyan', 'Julie',
                'Scott','Julie',]),
    ('Item Purchased',['Kit-Kat','Mango', 'Twix', 'M&Ms', 'Ferrero Rocher',
                       'Apple', 'Watermelon', 'Pineapple', 'Snickers','Pear','Kit-Kat','Mango', 'Twix', 'M&Ms', 'Ferrero Rocher',
                       'Apple']),
    ('Price Paid (USD)',  [1,2, 1.5, 2, 1.5,1,1,2, 1.5, 2,
                    1,2, 1.5, 2, 1.5,1]),
    ))
fav = DataFrame(table1)
fav

In [ ]:
#Insert Code Here

Pivots are a specific form of stack/unstack (remember those?)

![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/stack-unstack1.png "pivots")

## A worked example

In [ ]:
# to start let's make a fake dataset: sales of fruit across US states.
# Don't worry about the details here, but basically we'll pretend
# this string is a CSV file and use the standard loading ops
from io import StringIO

TESTDATA=StringIO("""State,Retailer,Fruit,Sales
MI,Walmart,Apple,100
MI,Wholefoods,Apple,150
MI,Kroger,Orange,180
CA,Walmart,Apple,220
CA,Wholefoods,Apple,180
CA,Safeway,Apple,220
CA,Safeway,Orange,110
NY,Walmart,Apple,90
NY,Walmart,Orange,80
NY,Wholefoods,Orange,120
""")

fruit = pd.read_csv(TESTDATA, index_col=None)
fruit

## (a) What is the total sales for each state?
This requires us to group by state, and aggregate sales by taking the sum.

The easiest way of doing this if to use `groupby`

If you execute groupby on the dataframe what you'll get back is an object called DataFrameGroupBy

In [ ]:
fruit.groupby('State')

On its own it's a bit useless... it just keeps track of which rows should go into each "pile" (where pile here means a unique group for each state)

If we ask this object to describe itself, you can see what is inside notice that it threw away all the other columns because they were not numerical.  Only "Sales" which is a number, was kept

In [ ]:
fruit.groupby('State').describe()

Now, if we had another numerical column, let's call it "Sales2," that column would also be kept.  Let's make a fruit2 DataFrame so you can see that:

In [ ]:
from io import StringIO

TESTDATA=StringIO("""State,Retailer,Fruit,Sales,Sales2
MI,Walmart,Apple,100,10
MI,Wholefoods,Apple,150,20
MI,Kroger,Orange,180,30
CA,Walmart,Apple,220,20
CA,Wholefoods,Apple,180,40
CA,Safeway,Apple,220,30
CA,Safeway,Orange,110,20
NY,Walmart,Apple,90,40
NY,Walmart,Orange,80,20
NY,Wholefoods,Orange,120,60
""")

fruit2 = pd.read_csv(TESTDATA, index_col=None)
fruit2

In [ ]:
fruit2.groupby("State").describe()

To actually make use of the groupby, we need to tell pandas what to use to measure what's in each group. In other words, I've created a pile for California, a pile for Michigan, and a pile for New York.  I want a number to what's *inside* each pile.  I could ask for the "size" (so how many rows are in each pile), or I could calculate some mathematical function.  For example, if I wanted to know the total sales, I would call "sum."  What happens is pandas goes through every pile, looks at every "row" inside that pile and, for all numerical properties, calculated something.  In this case it's sum... it adds up everything.  So in our original table we had three items for Michigan (Walmart, Kroger, Wholefoods).  This is our Michigan pile.  We then look at numerical properties for Walmart, Kroger, and Wholefoods. In this case Sales.  Because we are using sum() that means add the sales of each.

In [ ]:
# What are the total sales for each state?
fruit.groupby('State').sum()  # instead of size()

What just happend? A couple of things:
- `groupby()` got first executed on `df`, returning an `DataFrameGroupBy` object. This object itself is useless unless coupled with an aggregation function, such as `sum()`, `mean()`, `max()`, `apply()`. We will talk about `apply()` more in the next week.
- Then, `sum()` got executed on the `DataFrameGroupBy` object, generating the `DataFrame` object you see above. Notice how the table looks different than the original DataFrame `df`? Here are the differences:
  - The `State` column now becomes the index of the DataFrame. The string "State" is the name of the index. Notice how the index name is displayed on a lower level than column names.
  - Since we performed a `groupby` operation by `State`, so only the unique values of `State` are kept as index.
  - Among the other columns, Retailer, Fruit, and Sales, only Sales is kept in the result table. This is because the aggregation function `sum()` only knows how to aggregate numerical values. And only Sales is a numerical column. The other columns are hence dropped.

## <font color="magenta">Q11: (2 points) Using groupby, which Retailer had the highest total Sales and how much was that:

In [ ]:
# Insert code here

Insert written Answer here

## (b) What is the total sales for each state for each fruit?
This requires us to perform `groupby` on two columns. So, we provide a list of column names to the `groupby` function.

Don't forget that an aggregation function needs to follow the `groupby` function in order to generate results.

In [ ]:
# What is the total sales for each state for each fruit?
fruit.groupby(['State','Fruit']).sum()

How is this DataFrame different from the previous one?

The biggest different is that this DataFrame has what is called a `MultiIndex` (or hierarchical index), as opposed to a simple index. In this table, the left two "columns" are not columns but actually part of the `MultiIndex`, and the `Sales` is the single real "column" in the DataFrame. (Running out of terminologies here...)

The hierarchical index can be organized in an alternative way if we swapped the order of State and Fruit.

In [ ]:
fruit.groupby(['Fruit','State']).sum()

## (c) Which state has the maximum total sales?
This question is not asking about the maximum value, but rather which state holds that maximum. There are multiple ways to do it. A principled way is to use `idxmax`.

In [ ]:
# Which state has the maximum total sales?
fruitSalesByState = fruit.groupby('State').sum()
print(fruitSalesByState)
max_state = fruitSalesByState['Sales'].idxmax()
print("The state with the maximum sales is: ",max_state)



What if I want to display the maximum value alongside the state? Well, we can use that returned label to _select_ the corresponding row from the original DataFrame.

In [ ]:
# the deets
fruitSalesByState.loc['CA']

A less efficient but more intuitive way of doing the same thing:

In [ ]:
fruitSalesByState.sort_values('Sales',ascending=False).iloc[0]

### Which state has the maximum total sales for apples?</font>

In [ ]:
# Which state has the maximum total sales for apples?
# give me apple sellers
apples = fruit[fruit.Fruit == 'Apple']
apples

In [ ]:
# aggr. by state
applesByState = apples.groupby('State').sum()
applesByState

In [ ]:
applesByState.Sales

In [ ]:
applesByState.Sales.idxmax()

In [ ]:
applesByState.loc[applesByState.Sales.idxmax()]

In the above command, `.loc[]` looks up the index label and returns that row.



## Applying what we learned to the LOTR data

What are the average values for magic, aggression, and stealth for each race?

In [ ]:
from io import StringIO

LOTRDATA=StringIO("""name,race,gender,magic,aggression,stealth
Gandalf,Maia,Male,10.0,7.0,8.0
Gimli,Dwarf,None,1.0,10.0,2.0
Frodo,Hobbit,Male,4.0,2.0,5.0
Legolas,Elf,Male,6.0,5.0,10.0
Bilbo,Hobbit,Male,4.0,1.0,5.0
Sam,Hobbit,Male,2.0,6.0,4.0
Pippin,Hobbit,Male,0.0,3.0,5.0
Boromir,Human,Male,0.0,8.0,3.0
Aragorn,Human,Male,2.0,7.0,9.0
Galadriel,Elf,Female,9.0,2.0,10.0
Lily,Hobbit,Female,,,
Meriadoc,Hobbit,Male,,4.0,6.0
Melian,Maia,Female,10.0,5.0,9.0
Idril,Elf,Female,8.0,,8.0
""")

lotr = pd.read_csv(LOTRDATA, index_col=None)

In [ ]:
lotr.head()

## <font color="magenta">Q12: (2 points) Create a pivot table showing the maximum values of magic using gender for columns and race for rows.  

In [ ]:
#Insert Code Here

In [ ]:
lotr.groupby('race').mean()

We can make things a bit more complex by creating a dictionary of aggregations.  
The keys of the dictionary refer to the originating column we want to transform;
the values of the dictionary contain lists of the functions we want to apply to those columns.

Note that we can use built-in functions (see Table 10-1 in py4da) or define our own, possibly with a lambda:

In [ ]:
aggregations = {
    'magic': [min, max, sum, len],
    'aggression': [min, max, sum, len],
    'stealth': [min, max, lambda x: max(x) - min(x)]
}

Read more about DataFrame.agg(func, axis=0, *args, **kwargs) here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html

In [ ]:
grouped = lotr.groupby('race').agg(aggregations)
grouped

In [ ]:
grouped.columns

In [ ]:
grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]

In [ ]:
grouped.columns.ravel()

In [ ]:
grouped

### Which race has the lowest average magic value?

In [ ]:
lotr.groupby('race')['magic'].mean().idxmin()

# <font color="magenta">END OF NOTEBOOK</font>
## Remember to submit this file in HTML and IPYNB formats via Canvas.